Created 5.19.20

Authors: Margot Wagner, Sam Russman

COGS 260 Neural Data Analysis

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
from math import ceil

%matplotlib inline

# 1. Load dataset

#### Format data as matrix "X" of samples x features
- X starts as [n_chans x n_tp x n_samples x n_conditions] = [360 x 51 x 21 x 4]
- In this case we are splitting the baselines into two separate conditions

In [37]:
def load_data(filename, transpose=False):
    '''
    Load in data and return values
    
    param:   filename (str) - filename/path to data
             transpose - transposes data if true (default False)
    return:  data - data values (n_samples, n_features) 
    '''
    data = pd.read_csv(filename, header=None)
    
    # needs to be samples x features
    if transpose:    
        data = data.transpose()
        
    return data.values

In [38]:
'''Create X matrix'''
N_CHANS = 360
N_TIMEPTS = 51
N_SAMPLES = 21
N_CONDITIONS = 4
X = np.zeros((N_CHANS, N_TIMEPTS, N_SAMPLES, N_CONDITIONS))
L_COND_IDX = 0
R_COND_IDX = 1
NO_COND_IDX_1 = 2   # first half of baseline samples
NO_COND_IDX_2 = 3   # second half of baselines samples
IDX_SHIFT = 1       # index adjustments due to zero indexing

# 60mA stim and baseline
os.chdir('/Volumes/GoogleDrive/Shared drives/COGS 260 Project/Data/fragments3')
files = glob.glob('*60mA*') + glob.glob('baseline*')

# Sample numbers
l_i = 0
r_i = 0
no_i = 0
for filename in files:
    data = load_data(filename)
    
    # If data is from left condition, place in first condition index of X
    if '_l_' in filename:
        X[:, :, l_i, L_COND_IDX] = data
        l_i += 1    # increment left sample number
        
    # If data is from right condition, place in second condition index of X
    elif '_r_' in filename:
        X[:, :, r_i, R_COND_IDX] = data
        r_i += 1    # incremenet right sample number
        
    # If data is no stimulus condition, place in third condition index of X
    else:
        # Once condition 3 has 21 total samples, starting addition to condition 4
        if no_i <= (N_SAMPLES - IDX_SHIFT):
            X[:, :, no_i, NO_COND_IDX_1] = data
            no_i += 1    # increment no stim sample number
        else:
            X[:, :, no_i%(N_SAMPLES), NO_COND_IDX] = data
            no_i += 1    # increment no stim sample number
    
print("All data, X:", X.shape)   

All data, X: (360, 51, 21, 4)


#### Reshape to 2D matrix
- Reshape to X starts as [n_chans * n_tp x n_samples * n_conditions] = [360 * 51 x 21 * 4]
- After reshaping, we transpose the matrix, so the data matrix is of size n x p, where n is the number of samples and p is the number of variables/observations/features.
- We also want to center the data, so that the column/features means have been subtracted and are now equal to zero.


In [39]:
def reshape(X, transpose=True, center=True):
    '''
    Reshape, transpose, and center data matrix for PCA
    
    params
        X - data matrix
        transpose - if X needs to be transposed
        center - if X needs to be centers
        
    return
        X - data matrix, reshaped, transposed, and centered
    '''
    
    X = np.reshape(X, (N_CHANS*N_TIMEPTS, N_SAMPLES*N_CONDITIONS))
    
    if transpose:
        # Want X to be samples/observations x features
        X = X.T
        
    if center:
        # Remove column means
        X = X - np.mean(X, axis=0)
        
    return X
    

In [40]:
X = reshape(X)

# 2. Apply SVD
SVD gives X = USV^T where
U 
- [N_CHANS*N_TIMEPTS x N_CHANS*N_TIMEPTs (N_PCs)]
- Singular matrix containing orthogonal vectors of unit length in its row
    
S 
- [N_SAMPLES*N_CONDITIONS x 1]
V
- [N_SAMPLES*N_CONDITIONS x N_SAMPLES*N_CONDITIONS]
- Singular matrix containing orthogonal vectors of unit length in its column

V^T is the PC directions
US is the weights/loadings

In [66]:
'''Compute SVD of the original matrix'''
# Keep eigenvectors that have the top kth highest singular value (PCA)
u, s, vh = np.linalg.svd(X, full_matrices=False)

In [67]:
print(u.shape)
print(s.shape)
print(vh.shape)

(63, 63)
(63,)
(63, 18360)


In [47]:
vh.T

array([[ 5.16525547e-04, -4.31302146e-03,  3.81586738e-02, ...,
        -3.90897452e-02, -3.53526474e-02,  0.00000000e+00],
       [ 7.70798081e-04,  2.44676074e-02,  7.76103725e-02, ...,
        -1.40174834e-01,  4.34957876e-02,  0.00000000e+00],
       [ 8.15456195e-05,  1.41462006e-02,  1.40036837e-02, ...,
        -4.23021752e-02, -4.54717405e-02,  0.00000000e+00],
       ...,
       [-7.36374073e-05, -1.18223929e-02,  3.25543622e-02, ...,
        -2.62348791e-02, -9.86593758e-03,  0.00000000e+00],
       [ 8.41505831e-04,  8.51207465e-04,  3.45390859e-02, ...,
         1.52369890e-02, -3.53485195e-02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [77]:
from sklearn.decomposition import PCA
print(X.T.shape)
pca = PCA()
X_pca = pca.fit_transform(X.T)
pca.explained_variance_ratio_.shape

(18360, 63)


(63,)

### PCA 

In [194]:
from sklearn.decomposition import PCA
def run_pca(data):
    '''
    Does PCA on normalized data set. Can optionally set fewer components
    
    param: data - normalized data (n_samples, n_features)
    return: 
        data_pc: data transformed onto princinpal components
        components:  principal axes in feature space, array, shape (n_components, n_features)
        weights: percentage of variance explained by each of the selected components. array, shape (n_components,)
    '''
    # create PCA model
    pca = PCA() 
    
    # fit model to data
    data_pc = pca.fit(data)  
    
    # obtain components and components' weights
    components = pca.components_
    weights = pca.explained_variance_ratio_
    
    return data_pc, components, weights, pca

In [196]:
def cum_var_plot(weights, desired_var, plot=False):
    '''
    Cumulative variance plot (number of components vs cumulative variance captured) with calculated number 
    of PCs required to get to a certain desired variance explained
    
    params:
        weights: percentage of variance explained by each of the selected components. array, shape (n_components,)
        desired_var:  percent variance to find number of PCs for
    return
        pcs_req:   pcs required to captured at least desired variance
        captured_var   exact variance captured by pcs_req
    
    '''
    INDEX_SHIFT = 1
    # cumulative variance captured
    cum_var = np.cumsum(weights) 
    
    # find pcs req to get desired variance
    pcs_req = int(round(np.min(np.where(cum_var > desired_var)) + 1)) 
    
    # actual variance captured
    captured_var = cum_var[pcs_req-INDEX_SHIFT]
    
    # plotting
    if plot:
        plt.figure()
        plt.plot(range(INDEX_SHIFT,len(cum_var)+INDEX_SHIFT), cum_var)
        plt.axvline(x=pcs_req, ymin=0, ymax=1, color='k', linestyle='--')
        plt.xlabel('Number of components')
        plt.ylabel('Cumulative variance captured')
        plt.title('Cumulative Variance Captured by Principal Components')
    
    return pcs_req, captured_var

In [197]:
def biplot(data, pca, plot=False):
    '''
    Creates biplot for data mapping data onto top 2 or 3 principal components
    
    params
        data - normalized data, array shape (n_samples, n_features)
        pca - pca model
        
    return
        top_pcs[:,PC1_IDX] - pcs captured by top PC
        top_var[PC1_IDX] - variance captured by top PC
    '''
    PC1_IDX = 0    # index of first PC
    PC2_IDX = 1    # index of second PC
    DIM = 2        # dimensions to plot 
    
    # top 2 pc's and their variance explained
    top_pcs = pca.transform(data)[:,:DIM]
    top_var = pca.explained_variance_ratio_[:DIM]
    
    # plotting
    if plot:
        plt.figure()
        plt.scatter(top_pcs[:,PC1_IDX], top_pcs[:,PC2_IDX])
        plt.xlabel('PC1 ({:.1%})'.format(top_var[PC1_IDX]))
        plt.ylabel('PC2 ({:.1%})'.format(top_var[PC2_IDX]))
        plt.axvline(x=0, ymin=np.min(top_pcs[:,PC2_IDX]), ymax=np.max(top_pcs[:,PC2_IDX]), color='k', ls='--')
        plt.axhline(y=0, xmin=np.min(top_pcs[:,PC1_IDX]), xmax=np.max(top_pcs[:,PC1_IDX]), color='k', ls='--')
    
    return top_pcs[:,PC1_IDX], top_var[PC1_IDX]

# LDA Classification

In [295]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
def classify(data, clf):
    '''
    General classifier for reduced input data into 
    3 stimulus conditions (left, right, no stimulus)
    
    param:
        data - normalized and potentially transformed to pcs (n_samples, n_features)
    
    '''
    
    N_TRIALS = 21
    labels = np.array(['left', 'right', 'none'])
    labels = np.repeat(labels, N_TRIALS)
    labels = labels[:62]
    
    # TRANSFORM DATA TO LOWER DIM
    # Automatically sets test as 0.25
    # Random state so that it's the same every run
    X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state = 0)

    # train LDA classifier
    # look into using LDA for dimensionality reduction??
    model = clf.fit(X_train , y_train)
    y_predict = model.predict(X_test)
    
    # model accuracy for X_test
    acc = 100*accuracy_score(y_test, y_predict)
    print('Accuracy:',round(lda_acc,2),'%')
    print(classification_report(y_test, y_predict,target_names=['Left', 'Right', 'None']))
    
    # creating a confusion matrix
    cm = confusion_matrix(y_test, y_predict)
    print(cm)

### Receiver Operating Characteristic (ROC)

ROC curves have true positive (predicted positive and actually positive) rate on the Y axis and false positive (predicted positive, actually negative) on the X axis. THe top left corner for the plot is the "ideal" point with a false positive rate of zero and a true positive rate of one.  
  
In order to extend ROC curve and ROC area to multi-label classification, it is necessary to binarize the output (one vs all). One ROC curve can be drawn per label and/or one can draw a ROC curve by considering each element of the label indicator matrix as a binary prediction (micro-averaging).  
  
Another evaluation for multi-label classification is macro-averaging, which gives equal weight to the classification of each label.  
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#plot-roc-curves-for-the-multilabel-problem

In [ ]:
def roc_curve():
    #https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226
    #https://stackoverflow.com/questions/56090541/how-to-plot-precision-and-recall-of-multiclass-classifier/56092736
    #https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#in-multi-label-settings
    
    # Precision-Recall curve

### TODO: Add regularization step

### TODO: Add bootstrapping?

# Data Analysis

## Run PCA

In [234]:
''' Mounted shared Google Drive for data
Data is recordings from 360 good channels x time
Data should be samples x features (time x features)
360 features, 51 samples. '''
'''21 right, 21 left, 20 baseline'''
N_SAMPLES = 62
N_TIMEPOINTS = 51
N_EXTRA_DATA = 2

'''
Output is top pc for all time points (0:50), the variance explained by
top output(51), and the total number of pc's required for 80% variance
explained by number of samples (21 left, 21 right)'''
output = np.zeros([N_TIMEPOINTS + N_EXTRA_DATA, N_SAMPLES])

# 60mA stim and baseline
os.chdir('/Volumes/GoogleDrive/Shared drives/COGS 260 Project/Data/fragments3')
files = glob.glob('*60mA*') + glob.glob('baseline*')

trial_names = []    # initialize list of trials
index = 0      # initialize index

# Run PCA on all trials
for filename in files:
    if '60' in filename:
        trial_names.append((filename.split('_')[1] + '_trial_' 
                            + filename.split('_')[5]).split('.')[0])
    else:
        trial_names.append(filename.split('.')[0])
           
    # Initialize trial results
    trial_results = np.zeros([N_TIMEPOINTS + N_EXTRA_DATA])
    
    # Load and normalize data
    data = load_data(filename, transpose=True)
    data = norm_data(data)
    
    # Run pca
    data_pc, components, weights, pca = run_pca(data)
    
    # further analysis
    # plotting options turned off
    pcs_req, captured_var = cum_var_plot(weights, 0.8)
    top_pc, top_var = biplot(data, pca)

    # Organize results for single trial
    trial_results[:N_TIMEPOINTS] = top_pc
    trial_results[N_TIMEPOINTS] = top_var
    trial_results[N_TIMEPOINTS + 1] = pcs_req

    # Add to all trial output
    output[:, index] = trial_results

    # incremement index
    index += 1
    

In [ ]:
pd.DataFrame(output, columns=trial_names)

## Run LDA

Data is now timepoints x trials, and we are interested in using the timepoints as the features, so the data must be transformed. The data has to also be renormalized, so the mean and standard deviation for each timepoint feature is 0 and 1 respectively.

In [337]:
data = output[:N_TIMEPOINTS,:]
data = norm_data(data.transpose())

AttributeError: 'numpy.ndarray' object has no attribute 'corr'

In [ ]:
classify(data, LinearDiscriminantAnalysis())
classify(data, DecisionTreeClassifier(class_weight='balanced'))
classify(data, KNeighborsClassifier())

In [315]:
from sklearn.tree import DecisionTreeClassifier

N_TRIALS = 21
labels = np.array(['left', 'right', 'none'])
labels = np.repeat(labels, N_TRIALS)
labels = labels[:62]

# TRANSFORM DATA TO LOWER DIM
# Automatically sets test as 0.25
# Random state so that it's the same every run
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state = 0)

# train DTree
model = DecisionTreeClassifier(class_weight='balanced')
model.fit(X=X_train,y=y_train)
y_predict = model.predict(X_test)

# model accuracy for X_test
acc = 100*accuracy_score(y_test, y_predict)
print('Accuracy:',round(acc,2),'%')
print(classification_report(y_test, y_predict,target_names=['Left', 'Right', 'None']))

# creating a confusion matrix
cm = confusion_matrix(y_test, y_predict)
print(cm)


Accuracy: 62.5 %
              precision    recall  f1-score   support

        Left       0.40      0.50      0.44         4
       Right       0.75      0.75      0.75         4
        None       0.71      0.62      0.67         8

    accuracy                           0.62        16
   macro avg       0.62      0.62      0.62        16
weighted avg       0.64      0.62      0.63        16

[[2 0 2]
 [1 3 0]
 [2 1 5]]


In [319]:
classify(data, DecisionTreeClassifier(class_weight='balanced'))

Accuracy: 43.75 %
              precision    recall  f1-score   support

        Left       0.40      0.50      0.44         4
       Right       0.33      0.25      0.29         4
        None       0.75      0.75      0.75         8

    accuracy                           0.56        16
   macro avg       0.49      0.50      0.49        16
weighted avg       0.56      0.56      0.56        16

[[2 1 1]
 [2 1 1]
 [1 1 6]]


In [325]:
from sklearn.svm import SVC
# TRANSFORM DATA TO LOWER DIM
# Automatically sets test as 0.25
# Random state so that it's the same every run
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state = 0)

# train SVM classifier
model = SVC(kernel = 'linear', class_weight='balanced').fit(X_train,y_train)
y_predict = model.predict(X_test)

# model accuracy for X_test
acc = 100*accuracy_score(y_test, y_predict)
print('Accuracy:',round(acc,2),'%')
print(classification_report(y_test, y_predict,target_names=['Left', 'Right', 'None']))

# creating a confusion matrix
cm = confusion_matrix(y_test, y_predict)
print(cm)


Accuracy: 50.0 %
              precision    recall  f1-score   support

        Left       0.29      0.50      0.36         4
       Right       0.50      0.50      0.50         4
        None       0.80      0.50      0.62         8

    accuracy                           0.50        16
   macro avg       0.53      0.50      0.49        16
weighted avg       0.60      0.50      0.52        16

[[2 1 1]
 [2 2 0]
 [3 1 4]]


In [329]:
from sklearn.neighbors import KNeighborsClassifier
# TRANSFORM DATA TO LOWER DIM
# Automatically sets test as 0.25
# Random state so that it's the same every run
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state = 0)

# train SVM classifier
model = KNeighborsClassifier().fit(X_train,y_train)
y_predict = model.predict(X_test)

# model accuracy for X_test
acc = 100*accuracy_score(y_test, y_predict)
print('Accuracy:',round(acc,2),'%')
print(classification_report(y_test, y_predict,target_names=['Left', 'Right', 'None']))

# creating a confusion matrix
cm = confusion_matrix(y_test, y_predict)
print(cm)

Accuracy: 62.5 %
              precision    recall  f1-score   support

        Left       0.25      0.25      0.25         4
       Right       0.60      0.75      0.67         4
        None       0.86      0.75      0.80         8

    accuracy                           0.62        16
   macro avg       0.57      0.58      0.57        16
weighted avg       0.64      0.62      0.63        16

[[1 2 1]
 [1 3 0]
 [2 0 6]]


In [335]:
from sklearn.naive_bayes import GaussianNB
# TRANSFORM DATA TO LOWER DIM
# Automatically sets test as 0.25
# Random state so that it's the same every run
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state = 0)

# train SVM classifier
model = GaussianNB().fit(X_train,y_train)
y_predict = model.predict(X_test)

# model accuracy for X_test
acc = 100*accuracy_score(y_test, y_predict)
print('Accuracy:',round(acc,2),'%')
print(classification_report(y_test, y_predict,target_names=['Left', 'Right', 'None']))

# creating a confusion matrix
cm = confusion_matrix(y_test, y_predict)
print(cm)

Accuracy: 75.0 %
              precision    recall  f1-score   support

        Left       0.50      0.50      0.50         4
       Right       0.67      1.00      0.80         4
        None       1.00      0.75      0.86         8

    accuracy                           0.75        16
   macro avg       0.72      0.75      0.72        16
weighted avg       0.79      0.75      0.75        16

[[2 2 0]
 [0 4 0]
 [2 0 6]]


# Next steps

1. Clean up classifier function to generalize
2. Add bootstrapping to classifier function
3. Look at stim vs no stim separately from left vs right
4. Keep these results to show justification for splitting it up